In [342]:
from datetime import datetime
import pandas as pd
import locale
import warnings
import os

locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
warnings.filterwarnings("ignore")

In [343]:
def get_file_name() -> list[str]:
    file_list = []
    for file in os.listdir():
        if file.endswith('.xlsx') and 'Base Pagos' in file:
            file_list.append(file)
    return file_list

def get_last_date(dates_list: list[datetime]) -> datetime:
    last_date = dates_list[0]
    for date in dates_list:
        if date > last_date:
            last_date = date
    return last_date

def get_date(file_list: list[str]) -> str:
    dates_list = []
    for file in file_list:
        fecha_list =  file.split(' ')[-1].split('.')[:3]
        dates_list.append(datetime.strptime(fecha_list[0] + '.' + fecha_list[1] + '.' + fecha_list[2], '%d.%m.%Y'))
    return get_last_date(dates_list).strftime('%d.%m.%Y')

fecha = get_date(get_file_name())
fecha

'23.08.2024'

In [344]:
def validar_o_crear_carpeta() -> str:
    fecha_hoy = datetime.today().strftime('%Y.%m.%d')
    
    carpeta_reporte = 'reporte'
    
    carpeta_hoy = os.path.join(carpeta_reporte, fecha_hoy)
    
    if not os.path.exists(carpeta_hoy):
        os.makedirs(carpeta_hoy)
        print(f'Carpeta creada: {carpeta_hoy}')
    else:
        print(f'La carpeta ya existe: {carpeta_hoy}')
    
    return carpeta_hoy

file_path = validar_o_crear_carpeta()
file_path

La carpeta ya existe: reporte\2024.08.26


'reporte\\2024.08.26'

In [487]:
base_pagos_path = 'Base Pagos ' + fecha + '.xlsx'
asignacion_path = 'base_asignacion_20240813.xlsx'

print(base_pagos_path)
print(asignacion_path)

monoproducto = file_path + '/MONOPRODUCTO ' + fecha + '.xlsx'
multiproducto = file_path + '/MULTIPRODUCTO ' + fecha + '.xlsx'
reactiva = file_path + '/REACTIVA ' + fecha + '.xlsx'
no_enviados = file_path + '/NO ENVIADOS ' + fecha + '.xlsx'

mono_path = os.path.abspath(monoproducto)
multi_path = os.path.abspath(multiproducto)
react_path = os.path.abspath(reactiva)
no_enviados_path = os.path.abspath(no_enviados)

print(mono_path)
print(multi_path)
print(react_path)
print(no_enviados_path)

Base Pagos 23.08.2024.xlsx
base_asignacion_20240813.xlsx
c:\Users\miria\Documents\BBVA\GitHub\base-pagos\reporte\2024.08.26\MONOPRODUCTO 23.08.2024.xlsx
c:\Users\miria\Documents\BBVA\GitHub\base-pagos\reporte\2024.08.26\MULTIPRODUCTO 23.08.2024.xlsx
c:\Users\miria\Documents\BBVA\GitHub\base-pagos\reporte\2024.08.26\REACTIVA 23.08.2024.xlsx
c:\Users\miria\Documents\BBVA\GitHub\base-pagos\reporte\2024.08.26\NO ENVIADOS 23.08.2024.xlsx


In [347]:
#################### BASE DE PAGOS ####################

In [348]:
df_base = pd.read_excel(base_pagos_path)
print(df_base.shape)
print(df_base.dtypes)
df_base.head(5)

(289, 9)
Entidad         int64
FECHA          object
CODCEN          int64
CLAVSERV        int64
CENTROPAGO      int64
IMPORTE       float64
MONEDA         object
NOMBRE         object
ESTADO         object
dtype: object


,Entidad,FECHA,CODCEN,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO
0,11,2024-08-23,27853752,248,257,835.0,PEN,EMILIANO FEDERICO OSCCO PEZUA,P
1,11,2024-08-23,25003473,248,368,40.0,PEN,..L,P
2,11,2024-08-23,26798182,248,442,1200.0,PEN,],P
3,11,2024-08-23,24989851,248,368,400.0,PEN,A...,P
4,11,2024-08-23,21285186,248,321,200.0,PEN,ADA MIRIAM NIEVA SINCHE,P


In [349]:
fecha_formateada = pd.to_datetime('today').strftime('%d-%b')
fecha_formateada = fecha_formateada[:3] + fecha_formateada[3:].capitalize()

df_base['FECHA_ENVIO'] = fecha_formateada.replace('.', '')
df_base['ID_RESPONSABLE'] = 'MIV'

cols_base = ['Entidad', 'FECHA', 'CODCEN', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FECHA_ENVIO', 'ID_RESPONSABLE']
df_base = df_base[cols_base]
df_base = df_base.rename(columns={'CODCEN': 'CC'})

df_base['CC'] = df_base['CC'].astype('Int64').astype(str).str.zfill(8)
df_base['CC'] = df_base['CC'].str.replace(' ', '').str.replace(r'\D', '', regex=True).str[-8:]
df_base['CC'] = df_base['CC'].str.zfill(8)

df_base['CLAVSERV'] = df_base['CLAVSERV'].astype(str).str.zfill(5)

print(df_base.shape)
print(df_base.dtypes)
df_base.head(5)

(289, 11)
Entidad             int64
FECHA              object
CC                 object
CLAVSERV           object
CENTROPAGO          int64
IMPORTE           float64
MONEDA             object
NOMBRE             object
ESTADO             object
FECHA_ENVIO        object
ID_RESPONSABLE     object
dtype: object


,Entidad,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FECHA_ENVIO,ID_RESPONSABLE
0,11,2024-08-23,27853752,00248,257,835.0,PEN,EMILIANO FEDERICO OSCCO PEZUA,P,26-Ago,MIV
1,11,2024-08-23,25003473,00248,368,40.0,PEN,..L,P,26-Ago,MIV
2,11,2024-08-23,26798182,00248,442,1200.0,PEN,],P,26-Ago,MIV
3,11,2024-08-23,24989851,00248,368,400.0,PEN,A...,P,26-Ago,MIV
4,11,2024-08-23,21285186,00248,321,200.0,PEN,ADA MIRIAM NIEVA SINCHE,P,26-Ago,MIV


In [350]:
df_base.isnull().sum()

Entidad           0
FECHA             0
CC                0
CLAVSERV          0
CENTROPAGO        0
IMPORTE           0
MONEDA            0
NOMBRE            0
ESTADO            0
FECHA_ENVIO       0
ID_RESPONSABLE    0
dtype: int64

In [351]:
#################### ASIGNACION ####################

In [283]:
df_asignacion = pd.read_excel(asignacion_path)
print(df_asignacion.shape)
print(df_asignacion.dtypes)
df_asignacion.head(5)

(242511, 40)
codigo                        int64
contrato                      int64
cont_18                       int64
nombre                       object
clave                        object
producto                     object
DiasVenc                      int64
moneda                       object
capital                     float64
liqtotal                    float64
agencia                      object
Camp                        float64
Gestor                       object
numdoc                       object
oficina                      object
territorio                   object
Tramo                        object
codoficina                    int64
CAPITAL_SOLES               float64
IMP_CAMPAÑA                 float64
TIPO_FONDO                   object
FLAG_HONRADO                  int64
TIPO_CARTERA                 object
DESC_TRAMO                   object
Hibrido                      object
ESTUDIO ABOGADO              object
FECHA_MORA                   object
ID_CASTIGO     

,codigo,contrato,cont_18,nombre,clave,producto,DiasVenc,moneda,capital,liqtotal,...,ID_CONDONACION MORA_TEMP,ID_FRACCIONADO,ID_KOBSA_P.CANCELACION,ID_REPORTE_CANC,ID_CARTERA,ID_FLAG,OBSERVACION,AGENCIA CORRECTA,PAG LEGAL,PAG. INTER
0,30739481,110010629600055161,1100109600055161,DINA CHOQUE FERNANDEZ,BC,PRÉSTAMOS PYMES,257,PEN,76663.97,108418.69,...,-,-,-,-,STOCK,2,PAGO LEGAL AGOSTO // SE CORRIGIO,MORNESE MORA,1,-
1,30739481,110010619600057679,1100109600057679,DINA CHOQUE FERNANDEZ,BC,PRÉSTAMOS PYMES,268,PEN,177075.30,256680.65,...,-,-,-,-,STOCK,2,PAGO LEGAL AGOSTO // SE CORRIGIO,MORNESE MORA,1,-
2,21874900,110050295000360222,1100505000360222,JULIA ANGELICA SOTO COTRINA,BC,TARJETAS PYMES,307,PEN,23986.04,36966.81,...,-,-,-,-,STOCK,4,PAGO INTERBANCARIO // SE CORRIGIO,ASESCOM RJ,-,1
3,30293392,110130235002987960,1101305002987960,INNOVA FASHION GROUPSAC,BC,TARJETAS PYMES,215,PEN,142100.58,200698.07,...,-,-,-,-,STOCK,1,PAGO INTERBANCARIO // SE CORRIGIO,CLASA MORA,-,1
4,20687681,110174079600365104,1101749600365104,ELIZABETH HAYDEE CORDOVA AMAYA,BC,PRÉSTAMOS PYMES,356,PEN,200151.61,309435.56,...,-,1,-,-,STOCK,1,MARCA FRACCIONADO RETORNAR AGENCIA // SE CORRIGIO,RodriguezAngobaldo,-,-


In [352]:
#################### BACKUPS ####################

In [469]:
df_base_test = df_base.copy()
print('Base', df_base_test.shape)

df_asignacion_test = df_asignacion.copy()
print('Asignacion', df_asignacion_test.shape)

Base (289, 11)
Asignacion (242511, 40)


In [470]:
base_count = df_base_test.shape[0]
base_count

289

In [471]:
df_asignacion_test['codigo'] = df_asignacion_test['codigo'].astype('Int64').astype(str).str.zfill(8)

cols_asignacion = ['codigo', 'ID_FLAG', 'cont_18', 'nombre', 'TIPO_CARTERA', 'TIPO_FONDO', 'clave', 'AGENCIA CORRECTA', 'ID_VTA.']
df_base_test = df_base_test.merge(df_asignacion_test[cols_asignacion], left_on='CC', right_on='codigo', how='left')
df_base_test = df_base_test.drop(columns=['codigo'])
df_base_test = df_base_test.rename(columns={'Entidad':'ENTIDAD', 'ID_FLAG': 'FLAG', 'cont_18': 'CONTRATO', 'nombre': 'NOMBRE_CLIENTE', 'clave': 'CARTERA', 'AGENCIA CORRECTA': 'AGENCIA', 'ID_VTA.': 'ID_VTA'})

df_base_test.drop_duplicates(subset=['CC', 'IMPORTE', 'MONEDA', 'NOMBRE'], keep='first', inplace=True)

df_base_test['TIPO_CARTERA'] = df_base_test['TIPO_CARTERA'].fillna('NULL')
df_base_test['TIPO_FONDO'] = df_base_test['TIPO_FONDO'].fillna('NULL')

df_base_test['FLAG'] = df_base_test['FLAG'].astype('Int64')
df_base_test['CONTRATO'] = df_base_test['CONTRATO'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)
df_base_test['ENTIDAD'] = df_base_test['ENTIDAD'].astype('Int64').astype(str).str.zfill(4)

cols_base_test = ['ENTIDAD', 'FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'CONTRATO', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'TIPO_FONDO', 'CARTERA', 'AGENCIA', 'ID_VTA']
df_base_test = df_base_test[cols_base_test]

print(df_base_test.shape)
print(df_base_test.dtypes)
df_base_test.head(5)

(289, 19)
ENTIDAD            object
FECHA              object
CC                 object
CLAVSERV           object
CENTROPAGO          int64
IMPORTE           float64
MONEDA             object
NOMBRE             object
ESTADO             object
FLAG                Int64
CONTRATO           object
NOMBRE_CLIENTE     object
FECHA_ENVIO        object
ID_RESPONSABLE     object
TIPO_CARTERA       object
TIPO_FONDO         object
CARTERA            object
AGENCIA            object
ID_VTA             object
dtype: object


,ENTIDAD,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,TIPO_FONDO,CARTERA,AGENCIA,ID_VTA
0,0011,2024-08-23,27853752,00248,257,835.0,PEN,EMILIANO FEDERICO OSCCO PEZUA,P,1,001101839600385171,EMILIANO FEDERICO OSCCO PEZUA,26-Ago,MIV,UNSECURED,NULL,KSTBC,MORNESE MORA,-
1,0011,2024-08-23,25003473,00248,368,40.0,PEN,..L,P,1,001109395000374390,MARIO JEFFREY MORI FERNANDEZ,26-Ago,MIV,UNSECURED,NULL,EXJ,MORNESE MORA,-
2,0011,2024-08-23,26798182,00248,442,1200.0,PEN,],P,1,001103485000711084,ROBERTO FARROÐAN SANTISTEBAN,26-Ago,MIV,UNSECURED,NULL,EXJ,ASESCOM RJ,-
3,0011,2024-08-23,24989851,00248,368,400.0,PEN,A...,P,1,001108149602973630,JOHAN YVAN REVILLA LUNA,26-Ago,MIV,UNSECURED,NULL,KSTBC,MORNESE MORA,-
4,0011,2024-08-23,21285186,00248,321,200.0,PEN,ADA MIRIAM NIEVA SINCHE,P,2,001103215002499247,ADA MIRIAM NIEVA SINCHE,26-Ago,MIV,UNSECURED,NULL,KSTBC,MORNESE MORA,-


In [472]:
if base_count == df_base_test.shape[0]:
    print('REGISTROS OK')
elif base_count > df_base_test.shape[0]:
    print('Se han perdido registros')
else:
    print('Se han duplicado registros: ', df_base_test.shape[0] - base_count)

REGISTROS OK


In [473]:
df_base_test.shape

(289, 19)

In [474]:
df_base_no_flag = df_base_test[df_base_test['FLAG'].isnull()]
df_base_no_flag.shape

(11, 19)

In [475]:
no_flag_count = df_base_no_flag.shape[0]
no_flag_count

11

In [476]:
flag_count = base_count - no_flag_count
flag_count

278

In [365]:
#################### MONOPRODUCTO ####################

In [427]:
df_mono = df_base_test[df_base_test['FLAG'] == 1]
print(df_mono.shape)
df_mono.head(5)

(238, 19)


,ENTIDAD,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,TIPO_FONDO,CARTERA,AGENCIA,ID_VTA
0,0011,2024-08-23,27853752,00248,257,835.0,PEN,EMILIANO FEDERICO OSCCO PEZUA,P,1,001101839600385171,EMILIANO FEDERICO OSCCO PEZUA,26-Ago,MIV,UNSECURED,NULL,KSTBC,MORNESE MORA,-
1,0011,2024-08-23,25003473,00248,368,40.0,PEN,..L,P,1,001109395000374390,MARIO JEFFREY MORI FERNANDEZ,26-Ago,MIV,UNSECURED,NULL,EXJ,MORNESE MORA,-
2,0011,2024-08-23,26798182,00248,442,1200.0,PEN,],P,1,001103485000711084,ROBERTO FARROÐAN SANTISTEBAN,26-Ago,MIV,UNSECURED,NULL,EXJ,ASESCOM RJ,-
3,0011,2024-08-23,24989851,00248,368,400.0,PEN,A...,P,1,001108149602973630,JOHAN YVAN REVILLA LUNA,26-Ago,MIV,UNSECURED,NULL,KSTBC,MORNESE MORA,-
8,0011,2024-08-23,23020176,00248,847,600.0,PEN,ALBERT SANTIE BARRANTES ALVA,P,1,001105045005177180,ALBERT SANTIEBARRANTESALVA,26-Ago,MIV,UNSECURED,NULL,EXJ,ASESCOM RJ,-


In [428]:
mono_count = df_mono.shape[0]
mono_count

238

In [429]:
df_mono['TIPO_CARTERA'].value_counts()

TIPO_CARTERA
UNSECURED    228
SECURED        7
NULL           3
Name: count, dtype: int64

In [430]:
df_mono['TIPO_FONDO'].value_counts()

TIPO_FONDO
NULL        232
REACTIVA      6
Name: count, dtype: int64

In [431]:
df_mono[['TIPO_CARTERA', 'TIPO_FONDO']].value_counts()

TIPO_CARTERA  TIPO_FONDO
UNSECURED     NULL          224
SECURED       NULL            5
UNSECURED     REACTIVA        4
NULL          NULL            3
SECURED       REACTIVA        2
Name: count, dtype: int64

In [433]:
#################### MONO REACTIVA ####################

In [432]:
df_reactiva = df_mono[(df_mono['TIPO_CARTERA'] == 'UNSECURED') & (df_mono['TIPO_FONDO'] == 'REACTIVA')]
df_reactiva.drop(columns=['ENTIDAD', 'CENTROPAGO', 'ID_VTA'], inplace=True)
print(df_reactiva.shape)
df_reactiva.head(5)

(4, 16)


,FECHA,CC,CLAVSERV,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,TIPO_FONDO,CARTERA,AGENCIA
17,2024-08-23,23533740,00248,13000.0,PEN,AMERICA TRADING CENTER SAC///////,P,1,001101449600212843,AMERICA TRADING CENTER SAC,26-Ago,MIV,UNSECURED,REACTIVA,BC,MORNESE MORA
59,2024-08-23,25019714,00248,3000.0,PEN,CORPORACION BIOQUIMICA DEL SUR SAC,P,1,001106099600037771,CORPORACION BIOQUIMICA DEL SUR SAC,26-Ago,MIV,UNSECURED,REACTIVA,KSTBC,CLASA MORA
146,2024-08-23,28131889,00248,30000.0,PEN,INVERSIONES TURISTICAS RC SAC,P,1,001102499600941935,INVERSIONES TURISTICAS RC SAC,26-Ago,MIV,UNSECURED,REACTIVA,BC,CLASA MORA
296,2024-08-23,25876560,00248,800.0,PEN,TECNOLOGIA Y SUMINISTROS GARCIA EIRL.,P,1,001103069601019184,TEGNOLOGIA Y SUMINISTROS GARCIA EIRL,26-Ago,MIV,UNSECURED,REACTIVA,KSTBC,ASESCOM RJ


In [434]:
reactiva_count = df_reactiva.shape[0]
reactiva_count

4

In [435]:
df_reactiva.to_excel(reactiva, index=False)

In [436]:
#################### FIN REACTIVA ####################

In [437]:
df_no_mono = df_mono[(df_mono['TIPO_CARTERA'] != 'UNSECURED')]
no_mono_count = df_no_mono.shape[0]
no_mono_count

10

In [442]:
df_mono = df_mono[df_mono['TIPO_CARTERA'] == 'UNSECURED']
df_mono = df_mono[df_mono['TIPO_FONDO'] == 'NULL']
print(df_mono.shape)

(224, 19)


In [443]:
df_mono['CONTRATO'] = df_mono['CONTRATO'].astype(str).str.zfill(18)

In [444]:
df_mono.drop(columns=['ENTIDAD', 'CENTROPAGO', 'ID_VTA'], inplace=True)
print(df_mono.shape)
df_mono.head(5)

(224, 16)


,FECHA,CC,CLAVSERV,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,TIPO_FONDO,CARTERA,AGENCIA
0,2024-08-23,27853752,00248,835.0,PEN,EMILIANO FEDERICO OSCCO PEZUA,P,1,001101839600385171,EMILIANO FEDERICO OSCCO PEZUA,26-Ago,MIV,UNSECURED,NULL,KSTBC,MORNESE MORA
1,2024-08-23,25003473,00248,40.0,PEN,..L,P,1,001109395000374390,MARIO JEFFREY MORI FERNANDEZ,26-Ago,MIV,UNSECURED,NULL,EXJ,MORNESE MORA
2,2024-08-23,26798182,00248,1200.0,PEN,],P,1,001103485000711084,ROBERTO FARROÐAN SANTISTEBAN,26-Ago,MIV,UNSECURED,NULL,EXJ,ASESCOM RJ
3,2024-08-23,24989851,00248,400.0,PEN,A...,P,1,001108149602973630,JOHAN YVAN REVILLA LUNA,26-Ago,MIV,UNSECURED,NULL,KSTBC,MORNESE MORA
8,2024-08-23,23020176,00248,600.0,PEN,ALBERT SANTIE BARRANTES ALVA,P,1,001105045005177180,ALBERT SANTIEBARRANTESALVA,26-Ago,MIV,UNSECURED,NULL,EXJ,ASESCOM RJ


In [445]:
df_mono.to_excel(monoproducto, index=False)

In [447]:
mono_count_final = df_mono.shape[0]
mono_count_final

224

In [449]:
if mono_count == mono_count_final + reactiva_count + no_mono_count:
    print('MONOPRODUCTO OK')
else:
    print('MONOPRODUCTO ERROR')

MONOPRODUCTO OK


In [450]:
#################### FIN MONOPRODUCTO ####################

In [451]:
#################### MULTIPRODUCTO ####################

In [452]:
df_multi = df_base_test[df_base_test['FLAG'] > 1].copy()
print(df_multi.shape)
df_multi.head(5)

(40, 19)


,ENTIDAD,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,TIPO_FONDO,CARTERA,AGENCIA,ID_VTA
4,0011,2024-08-23,21285186,00248,321,200.0,PEN,ADA MIRIAM NIEVA SINCHE,P,2,001103215002499247,ADA MIRIAM NIEVA SINCHE,26-Ago,MIV,UNSECURED,NULL,KSTBC,MORNESE MORA,-
6,0011,2024-08-23,21285186,00248,321,220.0,PEN,ADA MIRIAM NIEVA SINCHE.,P,2,001103215002499247,ADA MIRIAM NIEVA SINCHE,26-Ago,MIV,UNSECURED,NULL,KSTBC,MORNESE MORA,-
18,0011,2024-08-23,26501114,00248,797,1000.0,PEN,AMPLIA TECNOLOGIA,P,2,001103109600894903,AMPLIA TECNOLOGIA SOCIEDAD ANONIMA CERRADA,26-Ago,MIV,UNSECURED,NULL,EXJ,CLASA MORA,-
27,0011,2024-08-23,22914580,00248,106,6000.0,PEN,ARGOS CONSULTING GROUP SAC 1588,P,2,001101065000681588,ARGOS CONSULTING GROUP SAC,26-Ago,MIV,UNSECURED,NULL,BC,ASESCOM RJ,-
29,0011,2024-08-23,22914580,00248,106,1000.0,PEN,ARGOS CONSULTING GROUP SAC 0589,P,2,001101065000681588,ARGOS CONSULTING GROUP SAC,26-Ago,MIV,UNSECURED,NULL,BC,ASESCOM RJ,-


In [453]:
multi_count = df_multi.shape[0]
multi_count

40

In [454]:
df_multi['TIPO_CARTERA'].value_counts()

TIPO_CARTERA
UNSECURED    40
Name: count, dtype: int64

In [455]:
df_multi['TIPO_FONDO'].value_counts()

TIPO_FONDO
NULL        39
REACTIVA     1
Name: count, dtype: int64

In [456]:
df_no_multi = df_multi[(df_multi['TIPO_CARTERA'] != 'UNSECURED')]
no_multi_count = df_no_multi.shape[0]
no_multi_count

0

In [457]:
df_multi = df_multi[df_multi['TIPO_CARTERA'] == 'UNSECURED']

df_multi['CONTRATO'] = None
df_multi['TIPO_FONDO'] = None
df_multi['CARTERA'] = None

df_multi = df_multi.drop(columns=['TIPO_CARTERA', 'AGENCIA'])
df_multi = df_multi.drop_duplicates(subset=['CC', 'IMPORTE', 'MONEDA', 'NOMBRE'])

cols_multi = ['FECHA', 'CC', 'CLAVSERV', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE']
df_multi = df_multi[cols_multi]

print(df_multi.shape)
df_multi.head(5)

(40, 14)


,FECHA,CC,CLAVSERV,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE
4,2024-08-23,21285186,00248,200.0,PEN,ADA MIRIAM NIEVA SINCHE,P,2,None,None,None,ADA MIRIAM NIEVA SINCHE,26-Ago,MIV
6,2024-08-23,21285186,00248,220.0,PEN,ADA MIRIAM NIEVA SINCHE.,P,2,None,None,None,ADA MIRIAM NIEVA SINCHE,26-Ago,MIV
18,2024-08-23,26501114,00248,1000.0,PEN,AMPLIA TECNOLOGIA,P,2,None,None,None,AMPLIA TECNOLOGIA SOCIEDAD ANONIMA CERRADA,26-Ago,MIV
27,2024-08-23,22914580,00248,6000.0,PEN,ARGOS CONSULTING GROUP SAC 1588,P,2,None,None,None,ARGOS CONSULTING GROUP SAC,26-Ago,MIV
29,2024-08-23,22914580,00248,1000.0,PEN,ARGOS CONSULTING GROUP SAC 0589,P,2,None,None,None,ARGOS CONSULTING GROUP SAC,26-Ago,MIV


In [458]:
df_multi.to_excel(multiproducto, index=False)

In [459]:
multi_count_final = df_multi.shape[0]
multi_count_final

40

In [460]:
if multi_count == multi_count_final + no_multi_count:
    print('MULTIPRODUCTO OK')
else:
    print('MULTIPRODUCTO ERROR')

MULTIPRODUCTO OK


In [461]:
#################### FIN MULTIPRODUCTO ####################

In [464]:
#################### NO ENVIADOS ####################

In [477]:
no_eviados = no_mono_count + no_multi_count + no_flag_count
enviados = mono_count_final + multi_count_final + reactiva_count

print('No enviados:', no_eviados)
print('Enviados:', enviados)
print('Total:', no_eviados + enviados)

No enviados: 21
Enviados: 268
Total: 289


In [478]:
if base_count == enviados + no_eviados:
    print('REGISTROS OK')
else:
    print('REGISTROS ERROR')

REGISTROS OK


In [479]:
cols_no_enviados = ['ENTIDAD', 'FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'TIPO_FONDO', 'TIPO_CARTERA', 'CONTRATO', 'ID_VTA']

In [481]:
df_no_mono = df_no_mono[cols_no_enviados]
df_no_multi = df_no_multi[cols_no_enviados]
df_base_no_flag = df_base_no_flag[cols_no_enviados]

print(df_no_mono.columns)
print(df_no_multi.columns)
print(df_base_no_flag.columns)

Index(['ENTIDAD', 'FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA',
       'NOMBRE', 'ESTADO', 'FLAG', 'TIPO_FONDO', 'TIPO_CARTERA', 'CONTRATO',
       'ID_VTA'],
      dtype='object')
Index(['ENTIDAD', 'FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA',
       'NOMBRE', 'ESTADO', 'FLAG', 'TIPO_FONDO', 'TIPO_CARTERA', 'CONTRATO',
       'ID_VTA'],
      dtype='object')
Index(['ENTIDAD', 'FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA',
       'NOMBRE', 'ESTADO', 'FLAG', 'TIPO_FONDO', 'TIPO_CARTERA', 'CONTRATO',
       'ID_VTA'],
      dtype='object')


In [483]:
df_no_enviados = pd.concat([df_no_mono, df_no_multi, df_base_no_flag])
df_no_enviados['CONTRATO'] = None
print(df_no_enviados.shape)
print(df_no_enviados.dtypes)
df_no_enviados.head(5)

(21, 14)
ENTIDAD          object
FECHA            object
CC               object
CLAVSERV         object
CENTROPAGO        int64
IMPORTE         float64
MONEDA           object
NOMBRE           object
ESTADO           object
FLAG              Int64
TIPO_FONDO       object
TIPO_CARTERA     object
CONTRATO         object
ID_VTA           object
dtype: object


,ENTIDAD,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,TIPO_FONDO,TIPO_CARTERA,CONTRATO,ID_VTA
11,0011,2024-08-23,21854138,00248,360,35000.0,PEN,ALEX YURI PEREZ PEREZ,P,1,NULL,SECURED,None,-
47,0011,2024-08-23,23066381,00248,178,2900.0,PEN,BRUNO ESPOSITO MARSAN DNI 25677934,P,1,NULL,SECURED,None,-
61,0011,2024-08-23,23080142,00248,140,2000.0,PEN,"CREACIONES CARISA SAC,",P,1,REACTIVA,SECURED,None,-
78,0011,2024-08-23,21328758,00248,249,4000.0,PEN,"DELIA JANNET ROJAS CASTILLO,",P,1,NULL,SECURED,None,-
90,0011,2024-08-23,22011704,00248,232,82.0,PEN,DNI 00495489,P,1,NULL,NULL,None,-


In [485]:
if no_eviados == df_no_enviados.shape[0]:
    print('NO ENVIADOS OK')
else:
    print('NO ENVIADOS ERROR')

NO ENVIADOS OK


In [495]:
df_no_enviados.to_excel(no_enviados_path, index=False)

In [496]:
#################### FORMATO EXCEL ####################

In [499]:
import openpyxl as op
from openpyxl.styles import Font, PatternFill, Alignment

def format_excel(file_path: str, validator: str) -> None:
    workbook = op.load_workbook(file_path)
    sheet = workbook.active
    
    # Definir estilos generales
    general_font = Font(name='Calibri', size=11)
    header_font = Font(name='Calibri', size=11, bold=True, color='FFFFFF')
    alignment_center = Alignment(horizontal='center', vertical='center')
    
    # Definir estilos específicos
    header_font_white = Font(name='Calibri', size=11, bold=True, color='000000')
    header_fill_blue = PatternFill(start_color='002060', end_color='002060', fill_type='solid')
    header_fill_yellow = PatternFill(start_color='FFD965', end_color='FFD965', fill_type='solid')
    header_fill_green = PatternFill(start_color='C4D79B', end_color='C4D79B', fill_type='solid')
    header_fill_orange = PatternFill(start_color='FABF8F', end_color='FABF8F', fill_type='solid')
    
    # Aplicar estilos generales a todas las celdas
    for row in sheet.iter_rows():
        for cell in row:
            cell.font = general_font
    
    # Aplicar estilos al encabezado (fila 1)
    for cell in sheet[1]:
        cell.font = header_font
        cell.alignment = alignment_center
    
    # Aplicar estilos específicos según el validador
    if validator == 'mono':
        for col in range(1, 10):  # Columnas A-I
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(10, 14):  # Columnas J-M
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(14, 16):  # Columnas N-O
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(16, 17):  # Columna P
            sheet.cell(row=1, column=col).fill = header_fill_orange
            sheet.cell(row=1, column=col).font = header_font_white
    
    elif validator == 'multi':
        for col in range(1, 10):  # Columnas A-I
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(10, 12):  # Columnas J-K
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(12, 15):  # Columnas L-N
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
    
    elif validator == 'react':
        for col in range(1, 10):  # Columnas A-I
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(10, 14):  # Columnas J-M
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(14, 16):  # Columnas N-O
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(16, 17):  # Columna P
            sheet.cell(row=1, column=col).fill = header_fill_orange
            sheet.cell(row=1, column=col).font = header_font_white
    
    elif validator == 'no_env':
        for col in range(1, 11): # Columnas A-J
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(11, 15): # Columnas K-N
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
    
    workbook.save(file_path)

In [500]:
format_excel(mono_path, "mono")
format_excel(multi_path, "multi")
format_excel(react_path, "react")
format_excel(no_enviados_path, "no_env")

os.startfile(mono_path)
os.startfile(multi_path)
os.startfile(react_path)
os.startfile(no_enviados_path)